In [1]:
import duckdb
import xml.etree.ElementTree as ET
# Import classifier
from ipynb.fs.full.group_classifier import predict_batch
print("Import successful!!")

/Users/marisbuttmann/PycharmProjects/stance-detection-german-llm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import successful
Import successful!!


In [2]:
# Connect to sql database
con = duckdb.connect(database='../data/database/german-parliament', read_only=False)

In [3]:
def preprocess_speech(speech:tuple) -> tuple[int, list[str]]:
    # We only need the speech it to later identify the origin speech
    speech_id = test_speech[0]
    text = test_speech[5]
    # Parse xml from string will throw ParseError if not parseable
    root = ET.fromstring(text)
    # We only need paragraphs -> this filters unecessary information, such as comments or applause
    paragraphs = root.findall('p')
    paragraphs_text = {"paragraphs":[]}
    # Get text of paragraphs
    for p in paragraphs:
        # Use this to filter out speaker information
        if p.attrib.get("klasse") == "redner":
            continue
        else:
            paragraphs_text["paragraphs"].append(p.text.strip()) #get text and remove potential irrelevant whitespaces

    return (speech_id, paragraphs_text)
    

In [8]:
test_speech = con.sql("select * from speech order by random() limit 1 ").fetchone()

In [9]:
speech_id, paragraphs = preprocess_speech(test_speech)

print(speech_id)
print(paragraphs)

279799
{'paragraphs': ['Eine Zusatzfrage, Herr Abgeordneter Wende.']}


In [3]:
paragraphs = {'paragraphs': ['Eine Zusatzfrage, Herr Abgeordneter Wende.']}
print(predict_batch(paragraphs))

Map: 100%|██████████| 1/1 [00:00<00:00, 408.44 examples/s]

Batch size: 1
[[('Eine', 'O'), ('Zusatzfrage', 'O'), (',', 'O'), ('Herr', 'B-EPPOL'), ('Abgeordneter', 'I-EPPOL'), ('Wende', 'O'), ('.', 'O')]]


In [ ]:
print(ET.tostring(root.find('name'), encoding='unicode'))